# WELCOME TO BISI LIBRARY 

In [12]:
# Import necessary modules
import datetime  # For handling dates and times
import pickle     # For saving and loading data
import os         # For interacting with the operating system (e.g., checking file existence)
import re         # For regular expressions in input validation

In [13]:
# Define constants for borrowing limits and due dates
MAX_BORROWED_BOOKS_REGULAR = 3    # Maximum number of books a regular user can borrow
MAX_BORROWED_BOOKS_PREMIUM = 5    # Maximum number of books a premium user can borrow
DAYS_BEFORE_DUE_DATE_REGULAR = 20 # Number of days until a borrowed book is due for regular users
DAYS_BEFORE_DUE_DATE_PREMIUM = 30 # Number of days until a borrowed book is due for premium users

In [14]:
class Book:
    """
    Represents a book in the library.
    """
    def __init__(self, title, author, isbn):
        self.title = title            # Title of the book
        self.author = author          # Author of the book
        self.isbn = isbn              # ISBN number of the book
        self.is_available = True      # Availability status (True means available)

    def __str__(self):
        """
        Returns a string representation of the book, including its availability.
        """
        status = "Available" if self.is_available else "Checked Out"
        return f"'{self.title}' by {self.author} (ISBN: {self.isbn}) - {status}"

In [15]:
class User:
    """
    Represents a user of the library.
    """
    def __init__(self, name, user_id):
        self.name = name                  # Name of the user
        self.user_id = user_id            # Unique ID of the user
        self.borrowed_books = []          # List to track borrowed books and their borrow/due dates

    def borrow_book(self, book):
        """
        Allows the user to borrow a book if they haven't reached their borrowing limit
        and if the book is available.
        """
        if len(self.borrowed_books) >= MAX_BORROWED_BOOKS_REGULAR:
            print(f"{self.name} has reached the borrowing limit of {MAX_BORROWED_BOOKS_REGULAR} books.")
            return False

        if book.is_available:
            book.is_available = False  # Mark the book as not available
            borrow_date = datetime.datetime.now()
            due_date = borrow_date + datetime.timedelta(days=DAYS_BEFORE_DUE_DATE_REGULAR)
            self.borrowed_books.append((book, borrow_date, due_date))  # Add book with borrow and due dates
            print(f"{self.name} borrowed '{book.title}' on {borrow_date.strftime('%Y-%m-%d %H:%M:%S')}. Due on {due_date.strftime('%Y-%m-%d %H:%M:%S')}.")
            return True
        else:
            print(f"'{book.title}' is currently not available.")
            return False

    def return_book(self, book):
        """
        Allows the user to return a borrowed book.
        """
        for borrowed_book, borrow_date, due_date in self.borrowed_books:
            if borrowed_book.isbn == book.isbn:
                self.borrowed_books.remove((borrowed_book, borrow_date, due_date))  # Remove the book from borrowed list
                book.is_available = True  # Mark the book as available

                return_date = datetime.datetime.now()
                if return_date > due_date:
                    print(f"{self.name} returned '{book.title}' on {return_date.strftime('%Y-%m-%d %H:%M:%S')} - Late!")
                else:
                    print(f"{self.name} returned '{book.title}' on {return_date.strftime('%Y-%m-%d %H:%M:%S')} - On time.")
                return True

        print(f"{self.name} did not borrow '{book.title}'.")
        return False

    def __str__(self):
        """
        Returns a string representation of the user and their borrowed books.
        """
        if self.borrowed_books:
            borrowed_titles = ', '.join([f"'{book.title}' (Due: {due_date.strftime('%Y-%m-%d')})" for book, _, due_date in self.borrowed_books])
        else:
            borrowed_titles = 'No books borrowed'
        return f"User: {self.name}, ID: {self.user_id}, Borrowed Books: {borrowed_titles}"

In [16]:
class PremiumUser(User):
    """
    Represents a premium user with a higher borrowing limit and extended borrowing duration.
    """
    def borrow_book(self, book):
        """
        Allows the premium user to borrow a book with a higher borrowing limit and extended due date.
        """
        if len(self.borrowed_books) >= MAX_BORROWED_BOOKS_PREMIUM:
            print(f"{self.name} has reached the borrowing limit of {MAX_BORROWED_BOOKS_PREMIUM} books.")
            return False
        return self.borrow_book_premium(book)  # Use the overridden method for premium users

    def borrow_book_premium(self, book):
        """
        Overrides the borrow_book method to set an extended due date for premium users.
        """
        if book.is_available:
            book.is_available = False  # Mark the book as not available
            borrow_date = datetime.datetime.now()
            due_date = borrow_date + datetime.timedelta(days=DAYS_BEFORE_DUE_DATE_PREMIUM)
            self.borrowed_books.append((book, borrow_date, due_date))  # Add book with borrow and due dates
            print(f"{self.name} (Premium) borrowed '{book.title}' on {borrow_date.strftime('%Y-%m-%d %H:%M:%S')}. Due on {due_date.strftime('%Y-%m-%d %H:%M:%S')}.")
            return True
        else:
            print(f"'{book.title}' is currently not available.")
            return False

    def get_membership_level(self):
        """
        Returns the membership level of the user.
        """
        return "Premium"

    def __str__(self):
        """
        Returns a string representation of the premium user and their borrowed books.
        """
        base_str = super().__str__()
        return f"{base_str} (Membership: {self.get_membership_level()})"

In [17]:
class Library:
    """
    Manages the collection of books and users in the library.
    """
    def __init__(self):
        self.books = []   # List to store all Book objects
        self.users = []   # List to store all User and PremiumUser objects
        self.load_data()  # Load existing data from files

    def add_book(self, book):
        """
        Adds a new book to the library.
        """
        self.books.append(book)
        print(f"Book '{book.title}' added to the library.")

    def add_user(self, user):
        """
        Registers a new user to the library.
        """
        self.users.append(user)
        print(f"User '{user.name}' registered in the library.")

    def find_book_by_isbn(self, isbn):
        """
        Searches for a book in the library by its ISBN.
        """
        # Validate ISBN format (starts with ISBN followed by digits)
        if not re.match(r'^ISBN\d+$', isbn):
            print("Invalid ISBN format. It should start with 'ISBN' followed by digits (e.g., ISBN123).")
            return None

        for book in self.books:
            if book.isbn == isbn:
                return book
        print(f"No book with ISBN {isbn} found in the library.")
        return None

    def find_user_by_id(self, user_id):
        """
        Searches for a user in the library by their user ID.
        """
        # Validate User ID format (alphanumeric)
        if not re.match(r'^[A-Za-z0-9]+$', user_id):
            print("Invalid User ID format. It should contain only letters and numbers.")
            return None

        for user in self.users:
            if user.user_id == user_id:
                return user
        print(f"No user with ID {user_id} found in the library.")
        return None

    def borrow_book(self, user_id, isbn):
        """
        Facilitates a user borrowing a book.
        """
        user = self.find_user_by_id(user_id)
        book = self.find_book_by_isbn(isbn)
        if user and book:
            user.borrow_book(book)

    def return_book(self, user_id, isbn):
        """
        Facilitates a user returning a book.
        """
        user = self.find_user_by_id(user_id)
        book = self.find_book_by_isbn(isbn)
        if user and book:
            user.return_book(book)

    def display_books(self):
        """
        Displays all books in the library along with their availability status.
        """
        print("\nLibrary Books:")
        if not self.books:
            print("No books available in the library.")
        else:
            for book in self.books:
                print(book)
        print()  # Add an empty line for better readability

    def display_users(self):
        """
        Displays all registered users and the books they have borrowed.
        """
        print("\nRegistered Users:")
        if not self.users:
            print("No users registered in the library.")
        else:
            for user in self.users:
                print(user)
        print()  # Add an empty line for better readability

    def save_data(self):
        """
        Saves the current state of books and users to files.
        """
        try:
            with open('books.dat', 'wb') as books_file:
                pickle.dump(self.books, books_file)
            with open('users.dat', 'wb') as users_file:
                pickle.dump(self.users, users_file)
            print("Library data has been saved.")
        except Exception as e:
            print(f"An error occurred while saving data: {e}")

    def load_data(self):
        """
        Loads the state of books and users from files if they exist.
        """
        try:
            if os.path.exists('books.dat'):
                with open('books.dat', 'rb') as books_file:
                    self.books = pickle.load(books_file)
                print("Books data loaded successfully.")
            else:
                print("No existing books data found. Starting with an empty library.")

            if os.path.exists('users.dat'):
                with open('users.dat', 'rb') as users_file:
                    self.users = pickle.load(users_file)
                print("Users data loaded successfully.")
            else:
                print("No existing users data found. Starting with no registered users.")
        except Exception as e:
            print(f"An error occurred while loading data: {e}")

    def shutdown(self):
        """
        Saves data and performs cleanup before shutting down the library system.
        """
        self.save_data()
        print("Library system is shutting down.")

In [18]:
# Create an instance of the Library
library = Library()

No existing books data found. Starting with an empty library.
No existing users data found. Starting with no registered users.


In [19]:
def format_isbn(isbn_input):
    """
    Formats the ISBN input by ensuring it starts with 'ISBN'.
    If the user enters only digits, 'ISBN' is prefixed automatically.
    
    Parameters:
        isbn_input (str): The ISBN input entered by the user.
    
    Returns:
        str: The formatted ISBN starting with 'ISBN'.
    """
    isbn_input = isbn_input.strip().upper()  # Remove leading/trailing spaces and convert to uppercase
    if not isbn_input.startswith('ISBN'):
        isbn_input = 'ISBN' + isbn_input
    return isbn_input

def display_menu():
    """
    Displays the main menu options.
    """
    print("\n=== Library Management System ===")
    print("1. Add a Book")
    print("2. Register a User")
    print("3. Borrow a Book")
    print("4. Return a Book")
    print("5. Display All Books")
    print("6. Display All Users")
    print("7. Save and Exit")

def add_book_interactive():
    """
    Interactively adds a new book to the library.
    """
    print("\n--- Add a New Book ---")
    title = input("Enter the book title: ").strip()
    author = input("Enter the book author(s): ").strip()
    isbn_input = input("Enter the book ISBN (e.g., 123): ").strip()
    isbn = format_isbn(isbn_input)  # Automatically prefix 'ISBN' if not present

    # Input validation
    if not title:
        print("Book title cannot be empty.")
        return
    if not author:
        print("Book author cannot be empty.")
        return
    if not re.match(r'^ISBN\d+$', isbn):
        print("Invalid ISBN format. It should start with 'ISBN' followed by digits (e.g., ISBN123).")
        return

    # Check if a book with the same ISBN already exists
    existing_book = library.find_book_by_isbn(isbn)
    if existing_book:
        print(f"A book with ISBN {isbn} already exists: {existing_book.title} by {existing_book.author}.")
        return

    # Create and add the new book
    new_book = Book(title, author, isbn)
    library.add_book(new_book)

def register_user_interactive():
    """
    Interactively registers a new user to the library.
    """
    print("\n--- Register a New User ---")
    name = input("Enter the user's name: ").strip()
    user_id = input("Enter the user's unique ID (alphanumeric): ").strip().upper()

    # Input validation
    if not name:
        print("User name cannot be empty.")
        return
    if not re.match(r'^[A-Za-z0-9]+$', user_id):
        print("Invalid User ID format. It should contain only letters and numbers (e.g., U001).")
        return

    # Check if a user with the same ID already exists
    existing_user = library.find_user_by_id(user_id)
    if existing_user:
        print(f"A user with ID {user_id} already exists: {existing_user.name}.")
        return

    # Determine if the user is regular or premium
    while True:
        user_type = input("Is the user a Premium User? (yes/no): ").strip().lower()
        if user_type in ['yes', 'y']:
            new_user = PremiumUser(name, user_id)
            break
        elif user_type in ['no', 'n']:
            new_user = User(name, user_id)
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")

    # Add the new user to the library
    library.add_user(new_user)

def borrow_book_interactive():
    """
    Interactively facilitates borrowing a book by a user.
    """
    print("\n--- Borrow a Book ---")
    user_id = input("Enter your user ID: ").strip().upper()
    isbn_input = input("Enter the ISBN of the book you want to borrow (e.g., 123): ").strip()
    isbn = format_isbn(isbn_input)  # Automatically prefix 'ISBN' if not present

    # Input validation
    if not user_id:
        print("User ID cannot be empty.")
        return
    if not re.match(r'^[A-Za-z0-9]+$', user_id):
        print("Invalid User ID format. It should contain only letters and numbers (e.g., U001).")
        return
    if not re.match(r'^ISBN\d+$', isbn):
        print("Invalid ISBN format. It should start with 'ISBN' followed by digits (e.g., ISBN123).")
        return

    library.borrow_book(user_id, isbn)

def return_book_interactive():
    """
    Interactively facilitates returning a book by a user.
    """
    print("\n--- Return a Book ---")
    user_id = input("Enter your user ID: ").strip().upper()
    isbn_input = input("Enter the ISBN of the book you want to return (e.g., 123): ").strip()
    isbn = format_isbn(isbn_input)  # Automatically prefix 'ISBN' if not present

    # Input validation
    if not user_id:
        print("User ID cannot be empty.")
        return
    if not re.match(r'^[A-Za-z0-9]+$', user_id):
        print("Invalid User ID format. It should contain only letters and numbers (e.g., U001).")
        return
    if not re.match(r'^ISBN\d+$', isbn):
        print("Invalid ISBN format. It should start with 'ISBN' followed by digits (e.g., ISBN123).")
        return

    library.return_book(user_id, isbn)

In [ ]:
def main_menu():
    """
    Displays the main menu and handles user input for navigating the library system.
    """
    while True:
        display_menu()  # Show the menu
        choice = input("Enter your choice (1-7): ").strip()

        if choice == '1':
            add_book_interactive()
        elif choice == '2':
            register_user_interactive()
        elif choice == '3':
            borrow_book_interactive()
        elif choice == '4':
            return_book_interactive()
        elif choice == '5':
            library.display_books()
        elif choice == '6':
            library.display_users()
        elif choice == '7':
            library.shutdown()
            break  # Exit the loop and terminate the program
        else:
            print("Invalid choice. Please enter a number between 1 and 7.")

# Start the interactive library management system
main_menu()


=== Library Management System ===
1. Add a Book
2. Register a User
3. Borrow a Book
4. Return a Book
5. Display All Books
6. Display All Users
7. Save and Exit
Enter your choice (1-7): 1

--- Add a New Book ---
Enter the book title: abc
Enter the book author(s): skp
Enter the book ISBN (e.g., 123): 23
No book with ISBN ISBN23 found in the library.
Book 'abc' added to the library.

=== Library Management System ===
1. Add a Book
2. Register a User
3. Borrow a Book
4. Return a Book
5. Display All Books
6. Display All Users
7. Save and Exit
